### Transformer from scratch

- https://peterbloem.nl/blog/transformers
- https://e2eml.school/transformers.html#attention

In [1]:
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
import torch.distributions as dist

import random, tqdm, sys, math, gzip
from dataclasses import dataclass

In [2]:
class SelfAttention(nn.Module):
    def __init__(self, emb, heads=8, mask=False):
        super().__init__()
        self.emb = emb  # embedding dimension
        self.heads = heads  # number of heads in multi-head attention
        self.mask = mask
        
        # to compute (q, k, v) for all heads. nn.Linear applies linear transformation
        self.tokeys = nn.Linear(emb, emb * heads, bias=False)
        self.toqueries = nn.Linear(emb, emb * heads, bias=False)
        self.tovalues = nn.Linear(emb, emb * heads, bias=False)
        
        # to combine heads
        self.unifyheads = nn.Linear(heads * emb, emb)

    def mask_(self, matrices, maskval=0.0, mask_diagonal=True):
        # Masks out all values in the given batch of matrices where i <= j holds, i < j if mask_diagonal is false
        b, h, w = matrices.size()
        indices = torch.triu_indices(h, w, offset=0 if mask_diagonal else 1)
        matrices[:, indices[0], indices[1]] = maskval

    def contains_nan(self, tensor):
        return bool((tensor != tensor).sum() > 0)

    def forward(self, x):  # computes scaled dot-product self-attention
        b, t, e = x.size()
        h = self.heads
        assert e == self.emb, f'Input embedding dim ({e}) should match layer embedding dim ({self.emb})'

        # compute (q, k, v) for all heads:
        #  1. self.to* funcs return vector sequences of embedding dimension `emb`
        #  2. these are cut into `h` chunks
        keys    = self.tokeys(x)   .view(b, t, h, e)
        queries = self.toqueries(x).view(b, t, h, e)
        values  = self.tovalues(x) .view(b, t, h, e)
        
        # Scaled dot-product self-attention
        
        # - fold heads into the batch dimension
        keys = keys.transpose(1, 2).contiguous().view(b * h, t, e)
        queries = queries.transpose(1, 2).contiguous().view(b * h, t, e)
        values = values.transpose(1, 2).contiguous().view(b * h, t, e)
    
        # - get dot product of queries and keys
        dot = torch.bmm(queries, keys.transpose(1, 2))
        dot = dot / math.sqrt(e)  # apply scaling
        
        assert dot.size() == (b*h, t, t), f'Matrix has size {dot.size()}, expected {(b*h, t, t)}.'
        
        if self.mask:
            self.mask_(dot, maskval=float('-inf'), mask_diagonal=False)
        
        dot = F.softmax(dot, dim=2) # dot now has row-wise self-attention probabilities
        
        assert not self.contains_nan(dot[:, 1:, :]) # only the first row may contain nan
        
        if self.mask == 'first':
            dot = dot.clone()
            dot[:, :1, :] = 0.0
        
        # apply the self attention to the values to get attention for each head
        out = torch.bmm(dot, values).view(b, h, t, e)
        
        # swap h, t back, unify heads
        out = out.transpose(1, 2).contiguous().view(b, t, h * e)
        return self.unifyheads(out)

In [3]:
class TransformerBlock(nn.Module):
    def __init__(self, emb, heads, mask, seq_length, ff_hidden_mult=4, dropout=0.0):
        super().__init__()
        self.attention = SelfAttention(emb, heads=heads, mask=mask)
        self.mask = mask
        # Layer normalization makes training faster
        self.norm1 = nn.LayerNorm(emb)
        self.norm2 = nn.LayerNorm(emb)
        # multiple hidden layers one after the other:
        self.ff = nn.Sequential(
            nn.Linear(emb, ff_hidden_mult * emb),
            nn.ReLU(),  # Rectified Linear Unit activation function
            nn.Linear(ff_hidden_mult * emb, emb)
        )
        # dropout is a regularization technique to make models less prone to overfitting
        # it randomly zeroes out neurons which suprisingly increases accuracy
        self.do = nn.Dropout(dropout)

    def forward(self, x):
        attention = self.attention(x)
        x = self.norm1(attention + x)
        x = self.do(x)
        ff = self.ff(x)
        x = self.norm2(ff + x)
        x = self.do(x)
        return x

In [4]:
class Transformer(nn.Module):
    def __init__(self, emb, heads, depth, seq_length, num_tokens):
        super().__init__()
        self.num_tokens = num_tokens
        # nn.Embedding is used for word embedding lookup
        self.token_embedding = nn.Embedding(embedding_dim=emb, num_embeddings=num_tokens)
        self.pos_embedding = nn.Embedding(embedding_dim=emb, num_embeddings=seq_length)

        self.unify_embeddings = nn.Linear(2*emb, emb)
        
        tblocks = []
        for i in range(depth):
            tblocks.append(
                TransformerBlock(emb=emb, heads=heads, seq_length=seq_length, mask=True))
        self.tblocks = nn.Sequential(*tblocks)
        
        self.toprobs = nn.Linear(emb, num_tokens)
        
    def forward(self, x):
        tokens = self.token_embedding(x)
        b, t, e = tokens.size()
        
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        positions = self.pos_embedding(torch.arange(t, device=device))[None, :, :].expand(b, t, e)
        x = self.unify_embeddings(torch.cat((tokens, positions), dim=2).view(-1, 2*e)).view(b, t, e)
        x = self.tblocks(x)
        x = self.toprobs(x.view(b*t, e)).view(b, t, self.num_tokens)
        # return predicted log-probability vectors for each token based on the preceding tokens
        return F.log_softmax(x, dim=2)

In [5]:
@dataclass
class TrainingArgs:
    num_batches: int = 1_000_000
    batch_size: int = 64
    data: str = "./data/enwik8.gz"
    tb_dir: str = "./runs"  # tensorboard logs
    lr: float = 0.0001  # learning rate
    embedding_size: int = 128
    num_heads: int = 8
    context: int = 32  # context length
    depth: int = 4
    seed: int = 1
    test_every: int = 1500
    test_subset: int = 100_000
    test_batchsize: int = 128
    gradient_clipping: float = 1.0
    lr_warmup: int = 5000

In [6]:
def sample(lnprobs, temperature=1.0):
    """
    Sample an element from a categorical distribution
    :param lnprobs: Outcome log-probabilities
    :param temperature: Sampling temperature. 1.0 follows the given distribution,
        0.0 returns the maximum probability element.
    :return: The index of the sampled element.
    """

    if temperature == 0.0:
        return lnprobs.argmax()

    p = F.softmax(lnprobs / temperature, dim=0)
    cd = dist.Categorical(p)

    return cd.sample()

def enwik8(path, n_train=int(90e6), n_valid=int(5e6), n_test=int(5e6)):
    """
    Load the enwik8 dataset from the Hutter challenge.
    Adapted from https://github.com/openai/blocksparse/blob/master/examples/transformer/enwik8.py
    :param path:
    :param n_train:
    :param n_valid:
    :param n_test:
    :return:
    """
    with gzip.open(path) if path.endswith('.gz') else open(path) as file:
        X = np.fromstring(file.read(n_train + n_valid + n_test), dtype=np.uint8)
        trX, vaX, teX = np.split(X, [n_train, n_train + n_valid])
        return torch.from_numpy(trX), torch.from_numpy(vaX), torch.from_numpy(teX)

In [46]:
NUM_TOKENS = 256
# Used for converting between nats and bits
LOG2E = math.log2(math.e)

def train(arg):
    torch.manual_seed(arg.seed)
    tbw = SummaryWriter(log_dir=arg.tb_dir)
    data_train, data_val, data_test = enwik8(arg.data)
    data_train, data_test = (torch.cat([data_train, data_val], dim=0), data_test)
    
    model = Transformer(emb=arg.embedding_size, heads=arg.num_heads, depth=arg.depth, seq_length=arg.context, num_tokens=NUM_TOKENS)
    if torch.cuda.is_available():
        model.cuda()
    opt = torch.optim.Adam(lr=arg.lr, params=model.parameters())
    
    for i in tqdm.trange(arg.num_batches):
        if arg.lr_warmup > 0 and i < arg.lr_warmup:
            lr = max(  (arg.lr / arg.lr_warmup) * i, 1e-10)
            opt.lr = lr
            
        opt.zero_grad()
        
        starts = torch.randint(size=(arg.batch_size, ), low=0, high=data_train.size(0) - arg.context - 1)
        seqs_source = [data_train[start  :start+arg.context  ] for start in starts]
        seqs_target = [data_train[start+1:start+arg.context+1] for start in starts]
        source = torch.cat([s[None, :] for s in seqs_source ], dim=0).to(torch.long)
        target = torch.cat([s[None, :] for s in seqs_target ], dim=0).to(torch.long)
        
        if torch.cuda.is_available():
            source, target = source.cuda(), target.cuda()
        source, target = Variable(source), Variable(target)
        
        output = model(source)
        
        loss = F.nll_loss(output.transpose(2, 1), target, reduction='mean')
        tbw.add_scalar('transformer/train-loss', float(loss.item()) * LOG2E, i * arg.batch_size)
        
        loss.backward()
        
        if arg.gradient_clipping > 0.0:
            nn.utils.clip_grad_norm_(model.parameters(), arg.gradient_clipping)
        
        opt.step()
        
        # test
        if i != 0 and (i % arg.test_every == 0 or i == arg.num_batches - 1):
            upto = data_test.size(0) if i == arg.num_batches - 1 else arg.test_subset
            data_sub = data_test[:upto]
            
            with torch.no_grad():
                bits, tot = 0.0, 0
                batch = [] # buffer, every time it fills up, we run it through the model

                for current in range(data_sub.size(0)):
                    fr = max(0, current - arg.context)
                    to = current + 1

                    context = data_sub[fr:to].to(torch.long)
                    if context.size(0) < arg.context + 1:
                        pad = torch.zeros(size=(arg.context + 1 - context.size(0),), dtype=torch.long)
                        context = torch.cat([pad, context], dim=0)

                        assert context.size(0) == arg.context + 1

                    if torch.cuda.is_available():
                        context = context.cuda()

                    batch.append(context[None, :])

                    if len(batch) == arg.test_batchsize or current == data_sub.size(0) - 1:

                        # batch is full, run it through the model
                        b = len(batch)

                        all = torch.cat(batch, dim=0)
                        source = all[:, :-1] # input
                        target = all[:, -1]  # target values

                        output = model(source)

                        device = 'cuda' if torch.cuda.is_available() else 'cpu'
                        lnprobs = output[torch.arange(b, device=device), -1, target]
                        log2probs = lnprobs * LOG2E # convert from nats to bits

                        bits += - log2probs.sum()
                        batch = [] # empty buffer

                bits_per_byte = bits / data_sub.size(0)

                # print validation performance. 1 bit per byte is (currently) state of the art.
                print(f'epoch{i}: {bits_per_byte:.4} bits per byte')
                tbw.add_scalar(f'transformer/eval-loss', bits_per_byte, i * arg.batch_size)

                # generate some random text
                GENSIZE = 600
                TEMP = 0.5
                seedfr = random.randint(0, data_test.size(0) - arg.context)
                input = data_test[seedfr:seedfr + arg.context].to(torch.long)

                if torch.cuda.is_available():
                    input = input.cuda()

                input = Variable(input)

                print('[', end='', flush=True)
                for c in input:
                    print(str(chr(c)), end='', flush=True)
                print(']', end='', flush=True)

                for _ in range(GENSIZE):
                    output = model(input[None, :])
                    c = sample(output[0, -1, :], TEMP)
                    print(str(chr(max(32, c))), end='', flush=True)

                    input = torch.cat([input[1:], c[None]], dim=0)

                print()
    return model

In [47]:
# Takes 12h+ on nvidia rtx 3060
arg_ = TrainingArgs()
model_ = train(arg_)

/tmp/ipykernel_2353/2631439982.py:29: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  X = np.fromstring(file.read(n_train + n_valid + n_test), dtype=np.uint8)
  0%|                                                                          | 1497/1000000 [00:40<5:28:26, 50.67it/s]

epoch1500: 3.031 bits per byte
[/128&lt;/small&gt;.

===Special ]in stire be in Hest of the state a [[Catername bout of a the of bet of a strat and reellect and the a contric trear a reling reand of which comple of [[turrie Chenturan]] the [[Imany a partically compurie prome booct fired the with rean of the and the prort of contratic for the of mact for of the pron the from the the in the are the or on a the compation in the port and of the a prond and a bother and the shact a the bechom be the [[Mear Aring Sone Airgran Aleran a of Sollent and of the he '''[[Dullican to portan Allage]] and a the [[erean extricial pared]] sure of a located in the the [[domat

  0%|                                                                        | 1501/1000000 [00:47<273:33:07,  1.01it/s]

  0%|▏                                                                         | 2996/1000000 [01:30<5:33:02, 49.89it/s]

epoch3000: 2.746 bits per byte
[ that is intended to simulate th]e comput by the [[Gologin Seventer]] in [[Marthrologine]] ([[Action of The Action 300]]), [[Forge distary of prigrams]] made have to the perchise of the surch by and and lew in the the word of the [[Aleria countryly]], [[suphertic of the succh of the Countin deserncest of the propentic with and the prover the comprover the conting the cars of [[England Restion]] while in the gener of rese and by the cond to compan a componse although the olock of the ongroups of [[Kapiran Aliston]] ([[Stert Amertion suppers]] ([[19939]]), [[Jeseral Firr Deng]], [[Monerra]], [[Englic Connorration]] (1999) and [

...

 17%|███████████▍                                                        | 168001/1000000 [1:28:44<292:47:10,  1.27s/it]

 17%|███████████▊                                                          | 169498/1000000 [1:29:15<4:36:04, 50.14it/s]

epoch169500: 1.865 bits per byte
[e the main mass of the magma con]tained the government of the community of the [[New York City|North America]] and [[Christian Army]], and [[John Leiber's Cooper Championship and Maron Book] * [http://www.guardian.co.uk/ The Stration of Albania Shah Honduras, and

 17%|███████████▊                                                          | 169498/1000000 [1:29:30<4:36:04, 50.14it/s]

 November 1960, the state of the distinct community and influenced a new theory (compare remaining the name that the first concept of the [[Leiber United States]] and [[Tourism]].  The bottom was also the signal of [[George Hall]] and [[Charles Various Marinee

In [48]:
torch.save(model_, './data/model.pt')

In [10]:
model_ = torch.load("./data/model.pt")

In [8]:
def sample_sequence(model, seed, max_context, length=600, temperature=0.5, verbose=False):
    """
    Sequentially samples a sequence from the model, token by token.
    :param model:
    :param seed: The sequence to start with.
    :param length: The total number of characters to sample.
    :param temperature: The sampling temperature.
    :param verbose: If true, the sampled sequence is also printed as it is sampled.
    :return: The sampled sequence, including the seed.
    """

    sequence = seed.detach().clone()

    if verbose: # Print the seed, surrounded by square brackets
        print('[', end='', flush=True)
        for c in seed:
            print(str(chr(c)), end='', flush=True)
        print(']', end='', flush=True)

    for _ in range(length):

        # Input is the tail end of the sampled sequence (as many tokens as the model can handle)
        input = sequence[-max_context:]

        # Run the current input through the model
        output = model(input[None, :])

        # Sample the next token from the probabilitys at the last position of the output.
        c = sample(output[0, -1, :], temperature)

        if verbose:
            print(str(chr(max(32, c))), end='', flush=True)

        sequence = torch.cat([sequence, c[None]], dim=0) # Append the sampled token to the sequence

    print()
    return seed

In [11]:
arg_ = TrainingArgs()
X = np.fromstring("Indian", dtype=np.uint8)
seed = torch.from_numpy(X).to(torch.long)
seed = seed.cuda()
_ = sample_sequence(model_, seed=seed, max_context=arg_.context, verbose=True, length=600)

[Indian]

/tmp/ipykernel_281/3937272662.py:2: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  X = np.fromstring("Indian", dtype=np.uint8)


 community of the [[Tony Life]], the [[Roman Catholic Church]] and the [[President of the United States|President of the United States]] [[Category:History of the American Revolution was considered to be seen the production of the same system in his status until the northeast of the state is normally distinctive for a [[musical sea]] and [[petroleum]], which are a property of the [[science fiction]] that would be possible to end in the same time.  The only large structure of the formation of the [[British Air Force]], [[British Free State]], [[British Empire]], [[Brazil]], [[Brazil]], [[Brazil
